# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2020-2021 учебный год

Илья Щуров, Соня Дымченко, Руслан Хайдуров, Павел Егоров, Максим Бекетов

# Семинар 10. Композиции алгоритмов, продолжение.

Сегодня мы познакомимся с алгоритмом ансамблирования решающих деревьев gradient boosting.


In [ ]:
import numpy as np
np.random.seed(42)

### Градиентный спуск

Градиентный спуск мы изучили на предыдщуих семинарах. Самый простой метод минимизации функции ошибки (квадратичной, например) в пространстве параметров (веса $w_i$ в линейной модели, например), для оптимизации в каждый момент времени двигаемся по антиградиенту функции с каким-то шагом $\eta$. 

$$w_{n+1} = w_n - \eta \cdot \frac{\partial f}{\partial w}$$

<img src="https://miro.medium.com/max/600/1*iNPHcCxIvcm7RwkRaMTx1g.jpeg">

### Градиентный бустинг

Теперь давайте представим, что на каждом шаге мы оптимизируем функционал ошибки $L(y, a(x))$ не в пространстве параметров алгоритма, а в пространстве функций, чтобы найти приближение $\hat f(x)$. Тогда на $N$-ом шаге, мы получим композицию алгоритмов (функций), которые также называют weak learners.

**Алгоритм**:

1. Первый алгоритм $b_0(X)$ предсказывает константу:
    $$a_0(x) = b_0(x) = \arg\min_c \sum_{k=1}^K L(y_k, c)$$
2. На каждом шаге $i = 1, \dots, N$:

    1) Вычисляем остатки
$$z^{i} = -\frac{\partial L(y, a_{i-1}(x))}{\partial a_{i-1}(x)}$$
    2) Обучаем модель $b_i(x)$ предсказывать $z^{i}$:
$$b_i(x) = \arg\min_{\theta_b} \sum_{k=1}^K(z_k^{i} - b_i(x_k))^2$$
    3) Подбираем коэффициент $\gamma_i$ одномерной минимизацией: 
$$\gamma_i = \arg\min_{\gamma}\sum_{k=0}^K L(y_k, a_{i-1}(x_k) + \gamma b_i(x_k)$$
    4) Дополняем композицию:
$$a_i = a_{i-1} + \gamma_i b_i(x)$$
3. Получаем итоговый алгоритм:
$$a_N(x) = \sum_{i=0}^N\gamma_i b_i(x)$$

[Визуализации](http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html)

## Реализуем градиентный бустинг для бинарной классификации

### Задание 1.

Выпишите формулу градиента функции ошибки - бинарной кросс-энтропии - по ответам модели $a(x) = \hat{y}$;

$$L(y, \hat(y)) = - y \cdot \log (\hat{y}) - (1-y) \cdot \log (1-\hat{y}) $$

$$\frac{\partial L(y, \hat y)}{\partial \hat y} = ?$$

Реализуйте полученную формулу в виде функции, возвращающей список из **анти**градиентов для каждого предсказания модели (длина выборки $K$).

In [ ]:
def loss_grad(y_true, y_pred):
    # YOUR CODE
    dloss = 0.
    # YOUR CODE
    return dloss 

In [ ]:
res = loss_grad(np.array([0, 0, 0, 0, 1, 1, 1]),
                np.array([0.2, 0.2, 0.5, 0.5, 0.2, 0.8, 0.8]))
assert np.all(np.isclose(res, np.array([-1.25, -1.25, -2., -2., -5., -1.25, -1.25])))

In [ ]:
res = loss_grad(np.array([0, 0, 0, 0, 1, 1, 1]),
                np.array([0., 0.2, 0.5, 0.5, 0.2, 1., 0.8]))
assert not any(np.isnan(res))

### Задание 2.

Реализуйте функцию, которая делает предсказание, принимая на вход матрицу признаков `X` (для которой делается предсказание), а также список обученных элементарных алгоритмов `estimators` и их веса `gammas`. 

Это нужно для шага 2.1, где мы вычисляем ответы уже построенного ансамбля, чтобы по ним посчитать антиградиент функции ошибки - остатки $z^i$.


In [ ]:
def predict(X, estimators, gammas):
    '''
    X: np.array of size KxM
    estimators: list with sklearn models of size N with .predict method
    gammas: list of size N
    
    return: np.array of size K
    '''
    # YOUR CODE
    y_pred = 
    # YOUR CODE
    return y_pred

### Задание 3.

Реализуйте функцию обучения, которая принимает на вход признаки `X` и метки `original_y`, а возвращает список из обученных базовых алгоритмов `estimators` и весов этих алгоритмов `gammas`. 

Используйте данную функцию `get_weight` для получения оптимального веса нового базового алгоритма в ансамбле.

In [ ]:
# Вспомогательная функция для нахождения веса \gamma нового базового алгоритма в ансамбле
# Это есть шаг 2.3

import numpy as np
from scipy.optimize import minimize_scalar


def get_weight(y, y_pred, y_prev_pred):
    """
    Решает задачу одномерной оптимизации (минимизации mse) с помощью scipy.optimize.minimize_scalar
    для нахождения оптимального веса gamma предсказаний нового алгоритма
    
    :param y: правильный ответ на объекте выборки
    :param y_pred: предсказание нового базового алгоритма на объекте выборки
    :param y_prev_pred: предсказание предыдущего ансамбля на этом объекте
    :return: optimal gamma
    """
    def mse(gamma, y, y_pred, y_prev_pred):
        """
        Рассчитывает ошибку для данного веса gamma
        нового предсказания y_pred

        """
        # YOUR CODE
        return 
    
    # YOUR CODE
    return 


In [ ]:
def fit_ensemble(X, y_true, n_estimators, base_estimator, lr):
    # Храните базовые алгоритмы тут
    estimators = []
    # А их веса здесь
    gammas = []
    
    for i in range(n_estimators):
        # Посчитайте градиент по предсказаниям текущего ансамбля
        # Шаг 2.1
        # YOUR CODE
        grad = # YOUR CODE
        
        # обучите базовый алгоритм
        # Шаг 2.2
        # YOUR CODE
        
        # получите его вес gamma
        # Шаг 2.3
        # YOUR CODE
        
        # сохраните результаты итерации        
        
    return estimators, gammas

### Теперь соберем из этого одну сущность, которая будет обучаться

Сущность, конечно, не идеальна

In [ ]:
class GBDT:
    def __init__(self, n_estimators, base_estimator, lr):
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.lr = lr
        self.estimators = []
        self.gammas = []
        
    def fit(self, X, y):
        # YOUR CODE
    
    def predict(self, X):
        # YOUR CODE

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X, y = make_classification(n_samples=600, n_features=2,
                           n_informative=2, n_redundant=0, n_repeated=0,
                           n_classes=2, n_clusters_per_class=2,
                           flip_y=0.05, class_sep=0.9, random_state=241)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=241)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
pred = dt.predict(X_test)
print('ROCAUC of simple Decision Tree:', roc_auc_score(y_test, pred))

In [ ]:
hyperparameters = {
    'n_estimators': 100,
    'base_estimator': DecisionTreeRegressor(),
    'lr': 0.05
}

gbdt = GBDT(**hyperparameters)
gbdt.fit(X_train, y_train)
pred = gbdt.predict(X_test)
print('ROCAUC of our Gradient Boosting:', roc_auc_score(y_test, pred))

In [ ]:
rf = RandomForestClassifier(n_estimators=hyperparameters['n_estimators'])
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
print('ROCAUC of Sklearn Random Forest:', roc_auc_score(y_test, pred))

In [ ]:
gb = GradientBoostingClassifier(n_estimators=hyperparameters['n_estimators'])
gb.fit(X_train, y_train)
pred = gb.predict(X_test)
print('ROCAUC of Sklearn Gradient Boosting:', roc_auc_score(y_test, pred))

Алгоритмы градиентного бустинга, но с модификациями, которые делают их в разы мощнее:
- [XGBoost](https://arxiv.org/abs/1603.02754) (оптимизации второго порядка)
- [LightGBM](http://www.audentia-gestion.fr/MICROSOFT/lightgbm.pdf) (особенное построение решающего дерева)
- [CatBoost](https://arxiv.org/abs/1810.11363) (особенные кодировки категориальных данных)

###  Визуализируем предсказания на тесте

In [ ]:
from matplotlib.colors import ListedColormap

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

def plot_surface(X, y, clf, ax):
    h = 0.2
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    ax.set_title(clf.__class__.__name__)
    ax.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
for clf, ax in zip([dt, gbdt, rf, gb], axes.ravel()):
    plot_surface(X_train, y_train, clf, ax)

С помощью градиентного бустинга также можно находить выбросы в данных; выбросами будут те точки, градиент на которых максимален по модулю.